# Hanks 

### Introduzione

Implementazione della teoria delle valenze di Patrick Hanks per costruire cluster semantici basati su un verbo specifico. Utilizzando un corpus e un parser sintattico, si estraggono i fillers per i ruoli di soggetto e oggetto del verbo, convertendoli in tipi semantici. Si richiede un corpus con almeno alcune centinaia di istanze del verbo.

L'esercitazione si compone dei seguenti step:
- Estrapolazione delle frasi contenenti il verbo scelto dal dataset.
- Parsing delle dipendenze della frase.
- Individuazione dei componenti dipendenti del verbo nell'albero di parsing, selezionando le dipendenze di tipo "nsubj" e "dobj" (soggetto e oggetto).
- Disambiguazione del soggetto e dell'oggetto utilizzando il WordNet e l'algoritmo di Lesk.
- Identificazione del supersenso (tipo semantico) sia per il soggetto che per l'oggetto.
- Calcolo delle frequenze delle coppie di supersensi (cluster semantici).
- Visualizzazione dei risultati ottenuti.

### Estrazione delle frasi
Per facilità, ho deciso di estrarre direttamente le frasi dal corpus SemCor utilizzando la libreria NLTK. La funzione "extract_sentences_with_word" prende in input una parola e un nome di file di output. Scorre tutte le frasi nel corpus SemCor e salva le frasi che contengono la parola specificata nel file di output. Ad esempio, ho utilizzato questa funzione per estrarre tutte le frasi che contengono la parola "see" e salvarle nel file "see_sentences.txt".

Ma poi mi sono accorto che conteneva frasi come 
- At the moment , Barco 's back was to the road so he did n't see the detectives close in on his convertible which , in their quest for the stolen lap rug , they proceeded to search .
- She it was who had looked to see if I was wearing shoes upon learning that I could n't drive .

Ho comunque scelto di utilizzare SemCore


In [4]:
import json
import nltk
from nltk.corpus import semcor, wordnet as wn
from nltk.corpus.reader.wordnet import Lemma
from nltk.corpus.reader.wordnet import Synset
from nltk.tree import Tree
# load all sentences from semcor_sentences.json by extracting the sentence which is the first element of each dictionary
def load_data():
    with open('semcor_sentences.json') as f:
        data = json.load(f)
    sentences = []
    for sentence in data:
        sentences.append(sentence[1])
    return sentences

sentences = load_data()

Nel file data_load.py è presente il codice che effettua l'estrazione delle frasi da SemCore:
- Estrae le frasi dal corpus SemCor che contengono una determinata parola (in questo caso, "see"). Questa parola viene prima lemmatizzata (cioè ridotta alla sua forma di base, come da verbo a infinito) e poi utilizzata per cercare le frasi corrispondenti ( a loro volta lemmatizzate in modo da verificare se la forma del verbo è presente, magari derivata in altri tempi)
- Il JSON contiene un indice associato ad ogni frase: si tratta dell'indice della frase nel SemCore corpus. Assieme alle altre funzioni disponibili può tornare utile per accedere alle informazioni semantiche sulla frase. 
- search_word_synset ha l'obiettivo di traversare la frase di SemCore taggata dall'info semantica ed estrarre il synset corrispondente. L'idea sarebbe quella di poterlo utilizzare
  per una verifica post esecuzione di Hanks. Ho però avuto diversi problemi di esecuzione, legati principalmente alla lentezza di processamento: non è possibile parallelizzare l'accesso a NLTK (lock sulle risorse) e quindi non ho potuto provare questa funzionalità se non per poche frasi.

### Parsing delle dipendenze della frase 

Recupero dei SUBJECT ed OBJECT del verbo. Viene utilizzato SpaCy.

In [5]:
import spacy
from spacy.lang.en import English
nlp = spacy.load("en_core_web_sm")

def extract_verb_subj_obj(sentence, token):
    """
    Extract the subject and object of a verb in a sentence
    :param sentence: a sentence
    :param verb: a verb
    :return: a list of two elements, the first is the subject and the second is the object
    """
    args = [c for c in token.children if c.dep_ in ('nsubj', 'dobj')]
    if len(args) == 2:
        if args[0].dep_ == 'nsubj' and args[1].dep_ == 'dobj':
            return args
        elif args[0].dep_ == 'dobj' and args[1].dep_ == 'nsubj':
            return [args[1], args[0]]
    return []

# provide an example
verb = 'see'
for sentence in sentences[:25]:
    doc = nlp(sentence)
    for token in doc:
        if token.lemma_ == verb and token.pos_ == 'VERB':
            args = extract_verb_subj_obj(doc, token)
            if len(args) > 0:
                print(args)
                print(doc)
              


[Department, fit]
`` This is one of the major items in the Fulton County general assistance program '' , the jury said , but the State Welfare Department `` has seen fit to distribute these funds through the welfare departments of all the counties in the state with the exception of Fulton County , which receives none of this money .
[who, sights]
Inheriting a more mature Mantle , who now has seen the sights on and off Broadway , Ralph Houk quietly bestowed , no pun intended , the mantle of authority on Mickey .
[Chicago, Ballet]
But barring a miracle , and do n't hold your breath for it , Chicago will not see the Leningrad-Kirov Ballet , which stems from the ballet cradle of the Maryinsky and is one of the great companies of the world .
[I, face]
I hear the boot of Lucifer , I see his filthy face '' !
[we, inculcation]
When we see the steady and methodical inculcation into humanity of the idea of man 's worthlessness - until redeemed - the necessity of the Devil may become evident as a

### Disambiguazione
Si utilizza Lesk. Approfondimenti e maggiori dettagli nell'esercitazione lab2_genus. 
Di seguito le utility legate al pre processing delle frasi
L'idea è utilizzare la frase in cui è contenuta la stessa parola come contesto di disambiguazione. Questo vale solo per OBJ e SUBJ

In [11]:
from collections import Counter
from nltk.wsd import lesk

def disambiguate_senses(word,sentence):
    """
    The function preprocesses the sentence into words, disambiguates the senses of the words using the Lesk algorithm,
    and returns a Counter object of the senses.
    """
    words = (sentence)
    # Disambiguate the senses of the words
    sense = lesk(context_sentence=words, ambiguous_word=word, pos='n')
    # Return a Counter object of the senses
    return sense


verb = 'see'
semantic_clusters = Counter()
semantic_clusters_data = {}

for sentence in sentences:
    doc = nlp(sentence)
    for token in doc:
        if token.lemma_ == verb and token.pos_ == 'VERB':
            args = extract_verb_subj_obj(doc, token)
            if len(args) > 0:
                subj_sense= (disambiguate_senses(args[0].text, sentence))
                obj_sense=(disambiguate_senses(args[1].text, sentence))
                 # provide the supersense of the sense
                subj_supersense = None
                obj_supersense =   None
                if subj_sense is not None and obj_sense is not None:
                    subj_supersense = subj_sense.lexname()
                    obj_supersense =   obj_sense.lexname()

                if (subj_supersense, obj_supersense) not in semantic_clusters_data:
                        semantic_clusters_data[(subj_supersense, obj_supersense)]= []
                semantic_clusters[(subj_supersense, obj_supersense)] += 1
                semantic_clusters_data[(subj_supersense, obj_supersense)].append(
                    {
                        'sentence':sentence,
                        'subject': args[0].text,
                        'object': args[1].text
                    })
                    
print("Semantic Clusters:")
for cluster, count in semantic_clusters.most_common(250):
    print("\nCluster:", cluster, "- Count:", count)
    print("First 5 sentences in the cluster:")
    cluster_data = semantic_clusters_data.get(cluster, [])
    _sentences = [data for data in cluster_data][:]
    for data in _sentences:
        sentence = data['sentence']
        subject = data['subject']
        obj = data['object']
        print("- Sentence:", sentence)
        print("  Subject:", subject)
        print("  Object:", obj)
        print()




Semantic Clusters:

Cluster: (None, None) - Count: 75
First 5 sentences in the cluster:
- Sentence: When we see the steady and methodical inculcation into humanity of the idea of man 's worthlessness - until redeemed - the necessity of the Devil may become evident as a weapon , a weapon designed and used time and time again in every age to whip men into a surrender to a particular church or church-state '' .
  Subject: we
  Object: inculcation

- Sentence: As a result , most people do n't have more than a vague idea what folklore actually is ; they see it as a potpourri of charming , moral legends and patriotic anecdotes , with a superstition or remedy thrown in here and there .
  Subject: they
  Object: it

- Sentence: Lucretius has remarked : `` The reason why all Mortals are so gripped by fear is that they see all sorts of things happening in the earth and sky with no discernable cause , and these they attribute to the will of God '' .
  Subject: they
  Object: sorts

- Sentence: Th

## Risultati

I risultati ottenuti dall'algoritmo di Hanks presentano alcune limitazioni:
- Molte istanze del verbo "see" hanno pronomi (come I, It, ecc.) come soggetto corrispondente. Questi pronomi non hanno una categoria lessicale diretta in WordNet e l'algoritmo Lesk restituisce None per i loro sensi oppure come meno spesso è capitato, viene assegnato "noun.substance o altri synset" probabilmente grazie al contesto. 
Questo porta a un tipo semantico non valido per questi casi.
- Nel caso in cui sia presente più volte il "verbo", Spice non identifica oggetto e soggetto correttamente e quindi come succede in questo caso (ma in molti altri).
  In alcuni casi avrei potuto effettuare altro processing sulla frase e magari splittare le frasi troppo lunghe.
- Altri errori sono legati a Spicy che non ha risolto la dipendenza sintattica, probabilmente a causa dell'ambiguità: 

Ex
- Cluster: ('noun.substance', 'noun.attribute') - Count: 1
- Sentence: He had seen a dry , old , yellowing hand reach out , with that painful solicitude , to touch , to rearrange , to shift aimlessly , some object worth a pfennig .
-  Subject: He     <---- ok 
-  Object: reach   <--- l'oggetto è hand (la content word) 
  

Come ulteriori approfondimenti che potrebbero portare a risultati migliori, si potrebbe considerare:
- Esplorare altre tecniche di disambiguazione dei sensi: Oltre all'algoritmo Lesk, è possibile valutare altre tecniche come l'Algoritmo di Lesk migliorato, l'Algoritmo di Wu-Palmer, l'Algoritmo di Babelfy o tecniche basate su Word Embeddings () come BERT o Word2Vec). Questi approcci offrono la possibilità di ottenere una maggiore precisione nella determinazione dei sensi delle parole.

Utilizzare risorse lessicali aggiuntive: Oltre a WordNet, si possono considerare altre risorse lessicali come FrameNet, VerbNet o ConceptNet.

Gestire i pronomi: Si potrebbero gestire con regole custom magari basate sul contesto, su regole grammaticali o su altre informazioni linguistiche per assegnare un senso appropriato. Ad esempio, se la frase è " I saw it flyng away ", si potrebbe supporre che 'it' faccia riferimento ad un animale/volatile e quindi associarlo a 'noun.animal'.È importante tenere presente i limiti e le peculiarità del dominio specifico in cui si opera, ma l'implementazione di regole custom può essere una soluzione praticabile. 